# This notebook compiles the Fish Island Data, does QA/QC filtering, and adds a custom variables, including a Day/Night parameter based on sun angle

In [1]:
import numpy as np
import pandas as pd
import datetime
import Data_Processor as DP

# D_path = 'C:/FishIsland_2017/'
D_path = 'Data/'

Out = 'G:/My Drive/FishIsland_Outputs/'

FluxFolder='EddyPro_Output/'
TenHz_Path = D_path+FluxFolder+'10Hz/eddypro_10Hz_full_output_2020-08-20T121146_exp.csv'
OneHz_Path = D_path+FluxFolder+'1Hz/eddypro_1Hz_full_output_2020-08-20T183544_exp.csv'
Met_Path = D_path+'CR1000_ClimateData.txt'
Soil_Path = D_path+'Soil_Data.csv'
PBLH = D_path+'NARR.csv'
Comp = DP.Compile([TenHz_Path,OneHz_Path],Met_Path,Soil_Path,PBLH,
                  Drop_Variables = [
    'daytime','DOY','file_records','used_records','none_flux',
    'qc_none_flux','none_v-adv','none_molar_density','none_mole_fraction' ,'none_mixing_ratio',
    'none_time_lag','none_spikes','none_var','w/none_cov','x_peak','x_offset','x_10%','x_30%','x_50%','x_70%','x_90%']
                 )
Drop = [
['2017-07-10 10:00:00','2017-07-10 15:00:00'],
['2017-07-12 12:0:00','2017-07-12 14:00:00'],
['2017-08-01 10:00:00','2017-08-01 14:00:00'],
['2017-08-21 11:00:00','2017-08-21 14:00:00'],
['2017-09-12 07:30:00','2017-09-12 15:00:00']
       ]
Comp.Date_Drop(Drop,'2017-06-23 00:00:00')    

Comp.StorageCorrection()
Comp.QC_Check(2)
Rm1 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('QC Filter Removal:\n',Rm1)
Comp.Rain_Check([7,0])
Rm2 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('Rain Filter Removal:\n',Rm2-Rm1)
Comp.Signal_Check(20)
Rm3 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('RSSI Filter Removal:\n',Rm3-Rm2)
Comp.Wind_Filter(30,35+180)
Rm4 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('Wind Filter Removal:\n',Rm4-Rm3)
Comp.Spike_Removal(z_thresh=4)
Rm5 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('Spike Filter Removal:\n',Rm5-Rm4)
Comp.Ustar_Drop(Override=.1)
Rm6 = 1- Comp.Data[['co2_flux','ch4_flux']].count()/Comp.Data[['co2_raw','ch4_raw']].count().values
print('u* Filter Removal:\n',Rm6-Rm5)
Comp.Wind_Bins(30)

Comp.CustomVars(LAT = 69.371182,LON = -134.880935,TZ = -6,Zm=2.87)

# Data = Comp.Data.copy()

print('Final Sample Size')

print(Data[['fco2','fch4']].count())

QC Filter Removal:
 co2_flux    0.045883
ch4_flux    0.108497
dtype: float64
Rain Filter Removal:
 co2_flux    0.000000
ch4_flux    0.002179
dtype: float64
ch4_noSSFilter    2041
ch4_flux          2041
dtype: int64
RSSI Filter Removal:
 co2_flux    0.000000
ch4_flux    0.133333
dtype: float64
Wind Filter Removal:
 co2_flux    0.083281
ch4_flux    0.071895
dtype: float64
Spike Filter Removal:
 co2_flux    0.041169
ch4_flux    0.033115
dtype: float64
u* Filter Removal:
 co2_flux    0.043683
ch4_flux    0.023094
dtype: float64
Final Sample Size
fco2    2501
fch4    1441
dtype: int64


C:\Users\User\FishIsland_Flux_Analysis\FluxProcessing\SunStatistics.py:25: RuntimeWarning: invalid value encountered in arccos
  W2 = np.degrees(np.arccos(np.cos(np.radians(90.833))/(np.cos(np.radians(LAT))*np.cos(np.radians(T2)))-np.tan(np.radians(LAT))*np.tan(np.radians(T2)))) #HA Sunrise (deg)


# Add day/night parameter and Save

In [4]:
print('Number of Records: ',Comp.Data.shape)


File_Name = 'FI_Data_2017_PostProcessing.csv'
Data.to_csv(D_path+File_Name)
print('Done')

Number of Records:  (3962, 306)
Done
